<a href="https://colab.research.google.com/github/chihchao/application-of-programming/blob/main/web_system_fastapi_uvicorn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os

# 建立 templates 資料夾
os.makedirs("templates", exist_ok=True)

# 建立 index.html
with open("templates/index.html", "w") as f:
    f.write("""
<!DOCTYPE html>
<html>
<head>
    <title>FastAPI + Jinja2 on Colab</title>
</head>
<body>
    <h1>Hello, {{ name }}!</h1>
</body>
</html>
    """)



In [ ]:
!pip install -q fastapi uvicorn jinja2 aiofiles pyngrok nest-asyncio

In [9]:
from fastapi import FastAPI, Request
from fastapi.templating import Jinja2Templates
from fastapi.responses import HTMLResponse

app = FastAPI()
templates = Jinja2Templates(directory="templates")

@app.get("/", response_class=HTMLResponse)
async def read_root(request: Request):
    return templates.TemplateResponse("index.html", {"request": request, "name": "World"})

@app.get("/hello", response_class=HTMLResponse)
async def say_hello(request: Request):
    params = dict(request.query_params)
    return templates.TemplateResponse("index.html", {"request": request, "name": params})

In [10]:
from google.colab import userdata
import nest_asyncio
import uvicorn
from pyngrok import ngrok

# 避免 Colab 中 asyncio 問題
nest_asyncio.apply()

# 使用 ngrok 建立 tunnel
ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# 啟動 uvicorn 伺服器
uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: NgrokTunnel: "https://9be9-35-245-94-240.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [376]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2001:b011:e605:d6e0:b13c:56a6:21dd:fb69:0 - "GET / HTTP/1.1" 200 OK
INFO:     2001:b011:e605:d6e0:b13c:56a6:21dd:fb69:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2001:b011:e605:d6e0:b13c:56a6:21dd:fb69:0 - "GET /hello HTTP/1.1" 200 OK
INFO:     2001:b011:e605:d6e0:b13c:56a6:21dd:fb69:0 - "GET /hello?name= HTTP/1.1" 200 OK
INFO:     2001:b011:e605:d6e0:b13c:56a6:21dd:fb69:0 - "GET /hello?name=18 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [376]


### 可以使用 threading 背景執行

```python
from pyngrok import ngrok
import uvicorn
import threading
from google.colab import userdata
# 建立 ngrok 公開網址
ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# 啟動 FastAPI app
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run_app).start()
```
